# Semantica y Vectores de palabra con Spacy

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion2/1-semantics-word-vectors.ipynb)

En este notebook exploraremos el concepto de word vectors utilizando la librería Spacy.

## Referencias
* [NLP - Natural Language Processing With Python](https://www.udemy.com/course/nlp-natural-language-processing-with-python)

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipykernel_1781537/2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt

In [5]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 56.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [6]:
import spacy

nlp = spacy.load('en_core_web_lg')

En esta ocasión, este modelo de Spacy más grande ya contiene vectores de palabras densos de 300 dimensiones que podemos utilizar de inmediato.

Por ejemplo, exploremos la palabra `lion`

In [7]:
nlp(u'lion').vector

array([ 1.8963e-01, -4.0309e-01,  3.5350e-01, -4.7907e-01, -4.3311e-01,
        2.3857e-01,  2.6962e-01,  6.4332e-02,  3.0767e-01,  1.3712e+00,
       -3.7582e-01, -2.2713e-01, -3.5657e-01, -2.5355e-01,  1.7543e-02,
        3.3962e-01,  7.4723e-02,  5.1226e-01, -3.9759e-01,  5.1333e-03,
       -3.0929e-01,  4.8911e-02, -1.8610e-01, -4.1702e-01, -8.1639e-01,
       -1.6908e-01, -2.6246e-01, -1.5983e-02,  1.2479e-01, -3.7276e-02,
       -5.7125e-01, -1.6296e-01,  1.2376e-01, -5.5464e-02,  1.3244e-01,
        2.7519e-02,  1.2592e-01, -3.2722e-01, -4.9165e-01, -3.5559e-01,
       -3.0630e-01,  6.1185e-02, -1.6932e-01, -6.2405e-02,  6.5763e-01,
       -2.7925e-01, -3.0450e-03, -2.2400e-02, -2.8015e-01, -2.1975e-01,
       -4.3188e-01,  3.9864e-02, -2.2102e-01, -4.2693e-02,  5.2748e-02,
        2.8726e-01,  1.2315e-01, -2.8662e-02,  7.8294e-02,  4.6754e-01,
       -2.4589e-01, -1.1064e-01,  7.2250e-02, -9.4980e-02, -2.7548e-01,
       -5.4097e-01,  1.2823e-01, -8.2408e-02,  3.1035e-01, -6.33

In [8]:
nlp(u'lion').vector.shape

(300,)

Como podemos observar, `lion` es representada por un vector de *300* dimensiones. Esto es lo que llamamos un `Word to Vector` o *Word2Vec*.

Sin embargo, esta no es la única característica de los modelos de Spacy. También podemos tener un vector de documento lo que significa que por cada documento vamos a promediar los vectores de cada vector que lo compone.

In [9]:
nlp(u'The quick brown fox jumped').vector.shape

(300,)

## Comprobando la similitud entre tokens

Recordemos que una de las características más importantes de estos vectores es que desbloquean exitosamente la semantica de las palabras con operaciones aritméticas, tarea que antes era muy difícil con las técnicas más clásicas.

Gracias a que ya contamos con vectores pre-computados, podemos medir la similitud de estas palabras. La similitud se mide con el coseno entre los vectores. 

El rango del coseno es $[-1,+1]$, donde valores cercanos a cero, indican que los vectores son ortogonales, es decir su ángulo es de 90 grados y se consideran disimilares. Cuando la similitud es positiva cercano a $1$, quiere decir que ambos vecrtores son similares, ya que su angulo es inferior a 90 grados. Finalmente, cuando la similitud es negativa, cercana a $-1$, quiere decir que los vectores son opuestos, su ángulo es mayor a 90 grados. Esto último es diferente a valores ceranos a 0 ya que lo opuesto también lleva consigo una relación semántica.

In [10]:
tokens = nlp('lion cat pet')

Por ejemplo, podemos establecer una relación semántica entre un león y un gato, amobos son felinos. Además, entre un gato y una mascota, ya que un gato es usualmente una. Ahora verifiquemos esta suposición aritméticamente:

In [11]:
def print_similarity(tokens):
    for token in tokens:
        sim = {t:token.similarity(t) for t in tokens}
        print(f'Similarities with word {token}:\n{sim}\n')

print_similarity(tokens)

Similarities with word lion:
{lion: 1.0, cat: 0.5265437960624695, pet: 0.39923766255378723}

Similarities with word cat:
{lion: 0.5265437960624695, cat: 1.0, pet: 0.7505456805229187}

Similarities with word pet:
{lion: 0.39923766255378723, cat: 0.7505456805229187, pet: 1.0}



Naturalmente, cada palabra es completamente similar con sigo misma, por eso para lion, vemos que la similitud es 1. Nótese que la palabra león tiene una similitud de 0.5 con la palabra gato, pero al mismo tiempo tiene una baja similitud con la palabra mascota. 

Entonces verdaderamente la similitud entre los vectores puede revelar una relación semántica entre ellos. Y lo mejor es que es sistemáticamente computable con aritmética vectorial.

Observemos otros ejemplos:

In [12]:
tokens = nlp('like love hate')

Desde nuestro conocimeinto, sabemos que `love` (amor) y `hate` (odio) son antonimos, es decir, palabras con significado opuesto. Pero como ambas tienden a ser utilizadas en el mismo contexto, sus vectores pueden tener algo de similitud. Esto significa que los vectores son buenos para detectar contexto pero no para obtener definiciones u obtener un significado humanamente entendible de las palabras.

In [13]:
print_similarity(tokens)

Similarities with word like:
{like: 1.0, love: 0.6579040288925171, hate: 0.6574652194976807}

Similarities with word love:
{like: 0.6579040288925171, love: 1.0, hate: 0.6393098831176758}

Similarities with word hate:
{like: 0.6574652194976807, love: 0.6393098831176758, hate: 1.0}



Ahora, podemos inspeccionar el vocabulario de Spacy:

In [14]:
nlp.vocab.vectors.shape

(342918, 300)

Tenemos un total de $342,918$ palabras con $300$ dimensiones cada una.

Si una palabra no está presente en el vocabulario, significa que no va a tener un vector.

In [15]:
tokens = nlp('dog cat nargle')
for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

dog True 7.0336733 False
cat True 6.6808186 False
nargle False 0.0 True


Esto debemos tenerlo en cuenta sobretodo a la hora de usar modelos pre-entrenados, si sabemos de antemano que nuestro corpus tiene tokens que no están en el vocabulario, pues lo más seguro es que nuestro modelo no sea lo suficientemente bueno.

## Calculando un nuevo vector
Técnicamente, podemos usar algebra lineal para obtener un vector de una nueva palabra. Veamos el ejemplo clásico de los word embeddings, la relación entre rey, hombre, mujer y reina:

In [16]:
from scipy import spatial

cosine_similarity = lambda v1, v2: 1 - spatial.distance.cosine(v1, v2)

In [ ]:
king = nlp.vocab['king'].vector
man = nlp.vocab['man'].vector
woman = nlp.vocab['woman'].vector

Ahora podemos realizar operaciones simples como adición y substracción de vectores y ver lo que nos deja.

Si a `king` le restamos `man` y le sumamos `woman`, quizás nuestra intuición nos diga que el resultado debería ser la palabra `reina` verdad?

In [26]:
# Notice for king we substract the man features and add the women features.
# We expect the resulting vector to be similar to Queen, princess, highness, etc.
nv = king - man + woman

In [37]:
cosine_similarity(nv, nlp.vocab['queen'].vector)

0.7880843808892162

In [42]:
computed_similarities = {word.text:cosine_similarity(nv, word.vector) 
                         for word in nlp.vocab if word.has_vector and word.is_lower and word.is_alpha
                        }


In [43]:
computed_similarities = sorted(computed_similarities.items(), key=lambda kv: kv[1], reverse=True)

In [44]:
computed_similarities[:10]

[('king', 0.802425971834419),
 ('queen', 0.7880843808892162),
 ('woman', 0.5150813201839692),
 ('she', 0.3956184274252026),
 ('lion', 0.3860151047392566),
 ('who', 0.3159499610158729),
 ('fox', 0.3040430463020818),
 ('brown', 0.28812391994746456),
 ('when', 0.2859679440155518),
 ('dare', 0.282605812365024)]

Observemos la medida se similitud para la palabra `queen`, is significativamente cerca a lo que queríamos!